## A tutorial on the new RV code

In [1]:
import numpy as np
import matplotlib
import os
import joblib
import datetime

In [2]:
# load rvm (this can be downloaded from paperdata repo)
rvm = joblib.load("/Users/cham/projects/sb2/data/v10_rvm.dump")
# select 200 templates
rvm = rvm.shrink(200)

@RVM: initializing Radial Velocity Machine (RVM)...


In [3]:
# synthesize test spectrum
test_wave = rvm.wave_mod[(rvm.wave_mod>5000-5)&(rvm.wave_mod<5300+5)]
test_flux = np.interp(test_wave, rvm.wave_mod*(1+123/299792.458), rvm.flux_mod[101])
test_fluxerr = test_flux*0+.03

In [4]:
# make cache
rvm.make_cache(cache_name="B", wave_range=(5000, 5300), rv_grid=(-1000, 1000, 10))

@RVM: making cache ...


### no acceleration

In [5]:
%%timeit -r 3
rvr = rvm.measure(test_wave, test_flux, cache_name=None, rv_grid=np.arange(-1000, 1001, 10))

8.06 s ± 28.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### vectorization without cache

In [6]:
%%timeit -r 3
rvr = rvm.measure(test_wave, test_flux, cache_name="matrix", rv_grid=np.arange(-1000, 1001, 10))

4.06 s ± 47 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### vectorization with cache

In [7]:
%%timeit -r 3
# note that the rv grid is set in *make_cache*, no longer to input here
rvr = rvm.measure(test_wave, test_flux, cache_name="B", nmc=50)

680 ms ± 4.37 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### vectorization with cache & bianry RV decomposition

In [8]:
%%timeit -r 3
rvr = rvm.measure_binary(test_wave, test_flux, cache_name="B", nmc=50)

/Users/cham/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:2149: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


749 ms ± 10.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Summary:

- If you have very small RAM (<4GB), do not use the cache option
- If you have large RAM, you can use **cache** method to speed up the RV measurements. It vectorizes the CCF evaluation on RV and template grid. It simply gives you about 10x speed.

Tips: the cache size depends on 
- the number of templates
- the spectral sampling rate of spectra
- the RV grid
Therefore, if you are working on a large number of templates, very dense RV grid, and high resolution spectra, be cautious! For LAMOST MRS spectra, the cache for the blue and red arms need about 2-3GB RAM.

